In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from datetime import datetime, date
from dateutil.relativedelta import relativedelta

from sklearn.preprocessing import StandardScaler

from math import ceil

from keras.callbacks import LambdaCallback
from keras.models import Sequential
from keras.layers import Dense, Activation
from keras.layers import LSTM
from keras.optimizers import RMSprop

%matplotlib inline

train=pd.read_csv('sales predictor/sales_train.csv')
test=pd.read_csv('sales predictor/test.csv')
submission = pd.read_csv('sales predictor/sample_submission.csv')
items = pd.read_csv('sales predictor/items.csv')
item_cats = pd.read_csv('sales predictor/item_categories.csv')
shops = pd.read_csv('sales predictor/shops.csv')
test_shops=test.shop_id.unique()
train=train[train.shop_id.isin(test_shops)]
test_items=test.item_id.unique()
train=train[train.item_id.isin(test_items)]
num_month = train['date_block_num'].max()+1
val=[]

Using TensorFlow backend.


In [2]:
train_clean = train.drop(labels = ['date', 'item_price'], axis = 1)
train_clean.head()

,date_block_num,shop_id,item_id,item_cnt_day
0,0,59,22154,1.0
10,0,25,2574,2.0
11,0,25,2574,1.0
12,0,25,2574,1.0
13,0,25,2574,2.0


In [3]:
train_clean = train_clean.groupby(["item_id","shop_id","date_block_num"]).sum().reset_index()
train_clean = train_clean.rename(index=str, columns = {"item_cnt_day":"item_cnt_month"})
train_clean = train_clean[["item_id","shop_id","date_block_num","item_cnt_month"]]
train_clean

,item_id,shop_id,date_block_num,item_cnt_month
0,30,2,2,1.0
1,30,2,5,1.0
2,30,2,15,1.0
3,30,2,16,1.0
4,30,3,1,4.0
...,...,...,...,...
600154,22167,58,14,1.0
600155,22167,58,20,1.0
600156,22167,59,9,1.0
600157,22167,59,11,2.0


In [12]:
dataset = train_clean.pivot_table(index = ['shop_id','item_id'],values = ['item_cnt_month'],columns = ['date_block_num'],fill_value = 0,aggfunc='sum')

In [6]:
dataset

item_cnt_month                             ...                 \
date_block_num              0  1  2  3  4  5  6  7  8  9   ... 24 25 26 27 28   
shop_id item_id                                            ...                  
2       30                   0  0  1  0  0  1  0  0  0  0  ...  0  0  0  0  0   
        31                   0  4  1  1  0  0  0  0  0  0  ...  0  0  0  0  0   
        32                   0  0  0  0  0  0  0  0  0  0  ...  0  0  1  0  0   
        33                   1  0  0  0  0  0  0  0  0  0  ...  0  1  0  1  1   
        53                   0  0  0  0  0  0  0  0  0  0  ...  0  1  0  0  0   
...                        ... .. .. .. .. .. .. .. .. ..  ... .. .. .. .. ..   
59      22145                0  0  0  0  0  0  0  0  0  0  ...  0  0  0  0  0   
        22154                1  0  0  0  0  0  0  0  0  0  ...  0  0  0  0  0   
        22162                0  0  0  0  0  0  0  0  0  0  ...  0  9  4  1  1   
        22164                0  0  0  0  0  0  0  0  0  0  ...  0  2  1  2  0   
        22167                0  0  0  0  0  0  0  0  0  1  ...  0  0  0  0  0   

                                
date_block_num  29 30 31 32 33  
shop_id item_id                 
2       30       0  0  0  0  0  
        31       0  0  0  0  1  
        32       0  0  1  0  0  
        33       0  1  0  1  0  
        53       0  0  0  0  0  
...             .. .. .. .. ..  
59      22145    0  0  0  0  0  
        22154    0  0  0  0  0  
        22162    0  0  1  0  0  
        22164    0  1  0  0  0  
        22167    0  0  0  0  0  

[111404 rows x 34 columns]

In [13]:
dataset.reset_index(inplace=True)

In [14]:
dataset

shop_id item_id item_cnt_month                       ...        \
date_block_num                              0  1  2  3  4  5  6  7  ... 24 25   
0                    2      30              0  0  1  0  0  1  0  0  ...  0  0   
1                    2      31              0  4  1  1  0  0  0  0  ...  0  0   
2                    2      32              0  0  0  0  0  0  0  0  ...  0  0   
3                    2      33              1  0  0  0  0  0  0  0  ...  0  1   
4                    2      53              0  0  0  0  0  0  0  0  ...  0  1   
...                ...     ...            ... .. .. .. .. .. .. ..  ... .. ..   
111399              59   22145              0  0  0  0  0  0  0  0  ...  0  0   
111400              59   22154              1  0  0  0  0  0  0  0  ...  0  0   
111401              59   22162              0  0  0  0  0  0  0  0  ...  0  9   
111402              59   22164              0  0  0  0  0  0  0  0  ...  0  2   
111403              59   22167              0  0  0  0  0  0  0  0  ...  0  0   

                                        
date_block_num 26 27 28 29 30 31 32 33  
0               0  0  0  0  0  0  0  0  
1               0  0  0  0  0  0  0  1  
2               1  0  0  0  0  1  0  0  
3               0  1  1  0  1  0  1  0  
4               0  0  0  0  0  0  0  0  
...            .. .. .. .. .. .. .. ..  
111399          0  0  0  0  0  0  0  0  
111400          0  0  0  0  0  0  0  0  
111401          4  1  1  0  0  1  0  0  
111402          1  2  0  0  1  0  0  0  
111403          0  0  0  0  0  0  0  0  

[111404 rows x 36 columns]

In [15]:
dataset=pd.merge(test,dataset,on=['shop_id','item_id'],how='left')

C:\Users\anupa\Anaconda3\lib\site-packages\pandas\core\reshape\merge.py:643: UserWarning: merging between different levels can give an unintended result (1 levels on the left,2 on the right)
  warnings.warn(msg, UserWarning)
C:\Users\anupa\Anaconda3\lib\site-packages\pandas\core\generic.py:3878: PerformanceWarning: dropping on a non-lexsorted multi-index without a level parameter may impact performance.
  obj = obj._drop_axis(labels, axis, level=level, errors=errors)


In [18]:
dataset.fillna(0,inplace=True)

In [19]:
dataset

,ID,shop_id,item_id,"(item_cnt_month, 0)","(item_cnt_month, 1)","(item_cnt_month, 2)","(item_cnt_month, 3)","(item_cnt_month, 4)","(item_cnt_month, 5)","(item_cnt_month, 6)",...,"(item_cnt_month, 24)","(item_cnt_month, 25)","(item_cnt_month, 26)","(item_cnt_month, 27)","(item_cnt_month, 28)","(item_cnt_month, 29)","(item_cnt_month, 30)","(item_cnt_month, 31)","(item_cnt_month, 32)","(item_cnt_month, 33)"
0,0,5,5037,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,2.0,0.0,0.0,0.0,1.0,1.0,1.0,3.0,1.0,0.0
1,1,5,5320,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,2,5,5233,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,3.0,2.0,0.0,1.0,3.0,1.0
3,3,5,5232,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
4,4,5,5268,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
214195,214195,45,18454,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,2.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
214196,214196,45,16188,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
214197,214197,45,15757,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
214198,214198,45,19648,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [20]:
dataset.drop(['shop_id','item_id'],inplace=True,axis=1)

In [28]:
dataset.drop(['ID'],axis=1,inplace=True)

In [29]:
X_train = np.expand_dims(dataset.values[:,:-1],axis = 2)
# the last column is our label
y_train = dataset.values[:,-1:]

# for test we keep all the columns execpt the first one
X_test = np.expand_dims(dataset.values[:,1:],axis = 2)

# lets have a look on the shape 
print(X_train.shape,y_train.shape,X_test.shape)

(214200, 33, 1) (214200, 1) (214200, 33, 1)


In [31]:
from keras.models import Sequential
from keras.layers import LSTM,Dense,Dropout

# our defining our model 
my_model = Sequential()
my_model.add(LSTM(units = 64,input_shape = (33,1)))
my_model.add(Dropout(0.4))
my_model.add(Dense(1))

my_model.compile(loss = 'mse',optimizer = 'adam', metrics = ['mean_squared_error'])
my_model.summary()
my_model.fit(X_train,y_train,batch_size = 4096,epochs = 10)

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_1 (LSTM)                (None, 64)                16896     
_________________________________________________________________
dropout_1 (Dropout)          (None, 64)                0         
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 65        
Total params: 16,961
Trainable params: 16,961
Non-trainable params: 0
_________________________________________________________________
Epoch 1/10
214200/214200 [==============================] - 57s 264us/step - loss: 30.5953 - mean_squared_error: 30.5953
Epoch 2/10
214200/214200 [==============================] - 55s 259us/step - loss: 30.2440 - mean_squared_error: 30.2440
Epoch 3/10
214200/214200 [==============================] - 57s 268us/step - loss: 30.0376 - mean_squared_error: 30.0376
Epoch 4/10
214200

In [32]:
submission_pfs = my_model.predict(X_test)

In [33]:
submission_pfs

array([[0.46338326],
       [0.11493658],
       [0.80311745],
       ...,
       [0.10889913],
       [0.11493658],
       [0.09783768]], dtype=float32)

In [34]:
submission = pd.DataFrame({'ID':test['ID'],'item_cnt_month':submission_pfs.ravel()})
# creating csv file from dataframe
submission.to_csv('sub_pfs.csv',index = False)